In [1]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Load Dataset
df = pd.read_csv("../data/synthetic_screen_time_500k.csv")

# Default Theme
theme_light = dbc.themes.BOOTSTRAP
theme_dark = dbc.themes.DARKLY

current_theme = theme_dark


ModuleNotFoundError: No module named 'jupyter_dash'

In [ ]:
# Initialize Jupyter Dash App
app = JupyterDash(__name__, external_stylesheets=[current_theme])
app.title = "Thesis Dashboard"


c:\Users\Vivke\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [ ]:
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H2("📊 Screen Time Thesis Dashboard", className="text-center mt-3"))
    ]),

    # Theme Switch Toggle
    dbc.Row([
        dbc.Col([
            html.Label("Theme Mode:"),
            dcc.RadioItems(
                options=[{"label":" Light", "value":"light"}, {"label":" Dark", "value":"dark"}],
                value="dark", id="theme-toggle", inline=True
            )
        ], width=3)
    ], className="mb-3"),

    # Tabs for Multi-Page Layout
    dcc.Tabs(id="tabs", value="overview", children=[
        dcc.Tab(label="Overview", value="overview"),
        dcc.Tab(label="Stress Analysis", value="stress"),
        dcc.Tab(label="Sleep Analysis", value="sleep"),
    ]),

    html.Div(id="tab-content", className="p-3")
], fluid=True)


In [ ]:
@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "value"), Input("theme-toggle", "value")]
)
def render_content(tab, theme):
    # Theme Handling (light/dark)
    global current_theme
    current_theme = theme_dark if theme == "dark" else theme_light

    # -------------------
    if tab == "overview":
        fig1 = px.histogram(df, x="daily_screen_time_hours", nbins=30, title="Screen Time Distribution")
        fig2 = px.box(df, x="gender", y="daily_screen_time_hours", color="gender", title="Gender-wise Screen Time")
        fig3 = px.scatter(df, x="daily_screen_time_hours", y="mental_health_score", color="gender",
                          trendline="ols", title="Screen Time vs Mental Health Score")
        return html.Div([
            dbc.Row([dbc.Col(dcc.Graph(figure=fig1), width=6),
                     dbc.Col(dcc.Graph(figure=fig2), width=6)]),
            dbc.Row([dbc.Col(dcc.Graph(figure=fig3), width=12)])
        ])

    elif tab == "stress":
        fig4 = px.violin(df, x="location_type", y="stress_level", color="location_type",
                         box=True, points="all", title="Stress Level by Location")
        fig5 = px.scatter(df, x="stress_level", y="daily_screen_time_hours", color="gender",
                          trendline="ols", title="Stress vs Screen Time")
        return html.Div([
            dbc.Row([dbc.Col(dcc.Graph(figure=fig4), width=6),
                     dbc.Col(dcc.Graph(figure=fig5), width=6)])
        ])

    elif tab == "sleep":
        fig6 = px.histogram(df, x="sleep_duration_hours", color="screen_time_group",
                            title="Sleep Duration by Screen Time Group")
        fig7 = px.scatter(df, x="sleep_duration_hours", y="mental_health_score",
                          color="screen_time_group", trendline="ols", title="Sleep vs Mental Health")
        return html.Div([
            dbc.Row([dbc.Col(dcc.Graph(figure=fig6), width=6),
                     dbc.Col(dcc.Graph(figure=fig7), width=6)])
        ])
